<a href="https://colab.research.google.com/github/RobsonVieiraSilv/LZ77/blob/main/LZ77.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Implementação do algoritmo Lempel-Ziv 77 (LZ77)**

In [54]:
def LZ77_compress(data, window_size):
    compressed_data = []
    i = 0

    while i < len(data):
        match_length = 0
        match_distance = 0
        caractere = data[i]

        # Define a janela de busca
        start_window = max(0, i - window_size)
        search_buffer = data[start_window:i]

        # Procura correspondência na janela de busca
        for j in range(len(search_buffer)):
            L = 0
            while (L < len(data) - i and
                   search_buffer[j:j+L+1] == data[i:i+L+1]):
                L += 1

            if L > match_length:
                match_length = L
                match_distance = len(search_buffer) - j

        if match_length == 0:
            compressed_data.append((0, 0, caractere))
            i += 1
        else:
            # Combina correspondências consecutivas, se possível
            if compressed_data and compressed_data[-1][0] == 1 and compressed_data[-1][1] == match_distance:
                compressed_data[-1] = (1, match_distance, compressed_data[-1][2] + match_length)
            else:
                compressed_data.append((1, match_distance, match_length))
            i += match_length

    return compressed_data

def LZ77_decompress(compressed_data):
    decompressed_data = []

    for F, P, L in compressed_data:
        if F == 0:
            decompressed_data.append(L)
        else:
            start = len(decompressed_data) - P
            for i in range(L):
                decompressed_data.append(decompressed_data[start + i])

    return ''.join(decompressed_data)

Primeiro teste: verificando se a string é convertida em uma sequência de tuplas idêntica a da sequência apresenta no livro

In [55]:
string = "ABBABBABBBAABABA"
# string = "ABABAABAAABBABAB"
# string = "1011010100010"

seq_tupla_0 = LZ77_compress(string, 4) ## 4 é o comprimento da janela, ou seja, window_size = 4
print(seq_tupla_0)

[(0, 0, 'A'), (0, 0, 'B'), (1, 1, 1), (1, 3, 6), (1, 4, 2), (1, 1, 1), (1, 3, 2), (1, 2, 2)]


Recuperação da string a partir da tupla gerada:

In [56]:
rec_string = LZ77_decompress(seq_tupla_0)
print(rec_string)

ABBABBABBBAABABA


Segundo teste: utilizar uma frase

In [57]:
texto = "O_rato_roeu_a_roupa_do_rei_de_Roma"
seq_tupla_1 = LZ77_compress(texto, 12)
seq_tupla_1

[(0, 0, 'O'),
 (0, 0, '_'),
 (0, 0, 'r'),
 (0, 0, 'a'),
 (0, 0, 't'),
 (0, 0, 'o'),
 (1, 5, 2),
 (1, 3, 1),
 (0, 0, 'e'),
 (0, 0, 'u'),
 (1, 10, 1),
 (1, 9, 1),
 (1, 7, 3),
 (1, 6, 1),
 (0, 0, 'p'),
 (1, 6, 2),
 (0, 0, 'd'),
 (1, 6, 1),
 (1, 9, 2),
 (0, 0, 'e'),
 (0, 0, 'i'),
 (1, 7, 2),
 (1, 4, 1),
 (1, 10, 1),
 (0, 0, 'R'),
 (1, 10, 1),
 (0, 0, 'm'),
 (0, 0, 'a')]

In [58]:
rec_texto = LZ77_decompress(seq_tupla_1)
rec_texto

'O_rato_roeu_a_roupa_do_rei_de_Roma'

**Codificação e decodificação com LZ77**

In [59]:
import math

def to_binary(n, length):
    return bin(n)[2:].zfill(length)

def LZ77_encoder(input_string, window_size):

    substrings = ['']
    tuplas = []

    i = 0
    while i < len(input_string):
        j = 1
        while input_string[i:i + j] in substrings and (i + j) <= len(input_string):
            j += 1
        j -= 1

        # Determina o índice da substring encontrada
        substring = input_string[i:i + j]

        pointer = substrings.index(substring)

        # Converte o ponteiro para binário com o número necessário de bits
        pointer_binary = to_binary(pointer, math.ceil(math.log2(len(substrings))))

        # Define o próximo bit que não está na substring para ser o bit extra
        next_bit = input_string[i + j] if (i + j) < len(input_string) else ''

        tuplas.append((pointer_binary, next_bit))

        if len(substrings) >= window_size:
            substrings.pop(0)
        substrings.append(substring + next_bit)

        i += j + 1

    encoded_string = []
    for i in range(len(tuplas)):
      if i==0:
        encoded_string.append(tuplas[i][1])
      else:
        encoded_string.append(tuplas[i][0])
        encoded_string.append(tuplas[i][1])

    return tuplas, ''.join(encoded_string)

In [60]:
def LZ77_decoder(encoded_string):
    substrings = ['']
    seq_tuplas = []
    input_string = []
    i = 0

    while i < len(encoded_string):
        # Determinar o número de bits do ponteiro (baseado no tamanho atual do dicionário)
        tupla_length = math.ceil(math.log2(len(substrings)))

        if tupla_length == 0:  # Caso inicial
            tupla = 0
        else:
            # Extrair o ponteiro em binário
            pointer_binary = encoded_string[i:i + tupla_length]
            tupla = int(pointer_binary, 2)
            i += tupla_length

        # Extrair o próximo bit (se existir)
        next_bit = encoded_string[i] if i < len(encoded_string) else ''
        i += 1

        # Adicionar a tuplas
        seq_tuplas.append((to_binary(tupla, tupla_length), next_bit))

        # Reconstruir o input_string
        substring = substrings[tupla] + next_bit
        input_string.append(substring)

        substrings.append(substring)

    input_string = ''.join(input_string)
    return seq_tuplas, input_string

Exemplo 1:

In [61]:
mensagem = "1011010100010"
encoded_tupla, encoded_msg = LZ77_encoder(mensagem, 12)
print("Sequência de tuplas:", encoded_tupla)
print("String codificada:", encoded_msg)

Sequência de tuplas: [('0', '1'), ('0', '0'), ('01', '1'), ('10', '1'), ('100', '0'), ('010', '0'), ('001', '0')]
String codificada: 100011101100001000010


In [62]:
seq_tupla_2, rec_mensagem = LZ77_decoder(encoded_msg)
print("Tuplas:",seq_tupla_2)
print("String decodificada:", rec_mensagem)

Tuplas: [('0', '1'), ('0', '0'), ('01', '1'), ('10', '1'), ('100', '0'), ('010', '0'), ('001', '0')]
String decodificada: 1011010100010


In [63]:
if rec_mensagem == mensagem:
  print(True)
else:
  print(False)

True


Exercício 7.6 do livro de David MacKay:
Encode the string **000000000000100000000000** using the basic Lempel-Ziv algorithm described above.

In [64]:
string_7_6 = "000000000000100000000000"
encoded_tupla_7_6, enconded_7_6 = LZ77_encoder(string_7_6, 12)
print("Tuplas:", encoded_tupla_7_6)
print("String codificada:", enconded_7_6)

Tuplas: [('0', '0'), ('1', '0'), ('10', '0'), ('11', '0'), ('010', '1'), ('100', '0'), ('110', '0')]
String codificada: 010100110010110001100


Exercício 7.7 do livro de David MacKay: Decode the string **00101011101100100100011010101000011** that was encoded using the basic Lempel-Ziv algorithm.

In [65]:
decode_7_7 = "00101011101100100100011010101000011"

string_7_7 = "0100001000100010101000001" ## resultado apresentado no livro

In [66]:
seq_tupla, mensagem_original = LZ77_decoder(decode_7_7)
print("Tuplas:", seq_tupla)
print("String decodificada:", mensagem_original)

Tuplas: [('0', '0'), ('0', '1'), ('01', '0'), ('11', '1'), ('011', '0'), ('010', '0'), ('100', '0'), ('110', '1'), ('0101', '0'), ('0001', '1')]
String decodificada: 0100001000100010101000001


In [67]:
if mensagem_original == string_7_7:
  print(True)
else:
  print(False)

True
